In [ ]:
import numpy as np
import theano
import theano.tensor as tt
import pymc3 as pm
import datetime
import pandas as pd

import time
import pickle

import matplotlib.pyplot as plt
import matplotlib
#theano.config.cxx = "clang++-mp-9.0"
theano.config.optimizer='fast_run'
theano.config.floatX = "float64"
 
#theano.config.cache_optimizations = True
# Legend-formating for matplotlib
def _format_k(prec):
    """
        format yaxis 10_000 as 10 k.
        _format_k(0)(1200, 1000.0) gives "1 k"
        _format_k(1)(1200, 1000.0) gives "1.2 k"
    """

    def inner(xval, tickpos):
        return f"${xval/1_000:.{prec}f}\,$k"

    return inner


In [ ]:
def plain_SIR_model(beta,gamma,N,S0,I0,R0,l=150):
    """Simple classical SIR-Model with time-constant beta,gamma"""
    def next_day(t,St,It,Rt,oS,oI,oR,eff,beta,gamma,N):
        eff = beta*St/N
        bSIoN = eff * It # -beta * (S*I)/N
        gI = gamma*It
        # Calculate differences
        dS = -bSIoN
        dI = bSIoN - gI
        dR = gI
        # Update values
        nxt_St = St + dS
        nxt_It = It + dI
        nxt_Rt = Rt + dR
        
        nxt_St = tt.clip(nxt_St,0,1e16) # Improves sampling stability
        
        # Pass on to next step
        return nxt_St,nxt_It,nxt_Rt, dS,dI,dR, eff
        
    # Initial internal state
    zero = tt.cast(0.,dtype='float64')
    initial_state = [S0,I0,R0,zero,zero,zero,zero]
    sequences = [tt.arange(l)]
    # Exceute the 'Loop'
    state,_ = theano.scan(fn=next_day,
                            sequences=sequences,# time dependent sequences can be passed as first params
                            outputs_info=initial_state,
                            non_sequences=[beta,gamma,N])
    return state


def Gompertz(a,b,c,l=150):
    t = tt.arange(l)
    return a*tt.exp(-b*tt.exp(-c*t))

In [ ]:
# Data from
#https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
#Germany,51.165690999999995,10.451526,
jhu_deaths = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,3,3,7,9,11,17,24,28,44,67,84,94,123,157,206,267,342,433,533,645,775,920,1107,1275,1444,1584,1810,2016,2349,2607,2767,2736,3022,3194,3294,3804,4052,4352,4459,4586,4862,5033,5279,5575,5760,5877,5976,6126,6314,6467,6623,6736,6812,6866,6993,6993,7275,7392,7510,7549,7569,7661,7738,7861,7884,7897,7938,7962,8003,8081,8144,8203,8228,8261,8283,8309,8372,8428,8470,8504,8530,8540,8555,8563,8602,8635,8658,8673,8685,8695,8736,8752,8772,8783,8793,8801,8807,8820,8851,8875,8887,8895,8895,8899,8914,8928,8940,8965,8968,8968,8976,8990,8995,9006,9010,9020,9023,9022,9032,9046,9057,9063,9070,9071,9074,9078,9080,9087,9088,9091,9092,9094,9099,9102,9110,9120,9124,9124,9125,9131,9135,9144,9147,9154,9154,9154,9163,9179,9181,9195,9201,9202,9203,9208,9213,9217,9230,9235,9235,9236,9241,9249,9263,9266,9272,9275,9276,9281,9285,9290,9290,9299]
jhu_ts = pd.date_range(datetime.date(2020,1,21),periods=len(jhu_deaths),freq='D')

# Limit to non-zero values, dropping the front of the list
first_non_zero_index = 47
jhu_deaths1 = jhu_deaths[first_non_zero_index:]
jhu_ts1 = jhu_ts[first_non_zero_index:]
print("first non zero deaths timestamp",jhu_ts1[0])

# Alternative Deaths from Situation-report graph of actual dates of deaths
rki_deaths_per_day = [1, 1, 1, 1, 5, 2, 3, 4, 11, 13, 12, 21, 40, 24, 42, 42, 67, 76, 92, 104, 107, 114, 172, 165, 162, 204, 199, 223, 240, 240, 254, 260, 257, 240, 246, 239, 243, 245, 227, 239, 241, 191, 203, 209, 172, 194, 157, 163, 129, 141, 132, 125, 128, 121, 106, 76, 92, 78, 93, 82, 69, 54, 63, 64, 63, 42, 54, 66, 47, 29, 40, 44, 45, 47, 31, 33, 27, 30, 31, 23, 21, 18, 19, 17, 18, 24, 15, 18, 18, 14, 8, 9, 8, 6, 11, 11, 14, 9, 6, 9, 7, 7, 9, 6, 3, 4, 4, 6, 7, 7, 7, 4, 7, 8, 1]
rki_deaths = np.cumsum(rki_deaths_per_day)
rki_ts = pd.date_range(datetime.date(2020,3,8),periods=len(rki_deaths),freq='D')

sources = {"jhu_deaths":(jhu_deaths,jhu_ts),"jhu1_deaths":(jhu_deaths1,jhu_ts1),"rki_deaths":(rki_deaths,rki_ts)}

In [4]:
# Include Symptomatic from arcgis-Dump
rki_df = pd.read_csv("data/RKI_COVID19_200723.csv", sep=",") 

def rdate(s):
    r = None
    if "T" in s:
        r = datetime.datetime.strptime(s.split("T")[0],"%Y-%m-%d")
    if "-" in s:
        r = datetime.datetime.strptime(s.split(" ")[0],"%Y-%m-%d")
    else:
        if s[:4] == "2020":
            r = datetime.datetime.strptime(s.split(" ")[0],"%Y/%m/%d")
        else:
            r = datetime.datetime.strptime(s.split(" ")[0],"%m/%d/%Y")
    return r.date()

cols = [x for x in ["Meldedatum","Refdatum"] if x in rki_df.columns]
for col in cols:
    if col in rki_df:
        rki_df[col] = rki_df[col].apply(rdate)
        
rki_df = rki_df.filter(items=["Meldedatum","Refdatum","NeuerFall","AnzahlFall","IstErkrankungsbeginn","AnzahlTodesfall"])

ts = pd.date_range(datetime.date(2020,1,1),periods=200,freq='D')
symptomatic_timeseries = pd.date_range(datetime.date(2020,2,1),periods=150,freq='D')

symptomatic = np.zeros(len(ts))
for i,t in enumerate(ts):
    m = rki_df[rki_df["Refdatum"] == t]
    symptomatic[i] = np.sum(m[m["IstErkrankungsbeginn"] == 1]["AnzahlFall"])

cs = np.cumsum(symptomatic)
s = cs[(symptomatic_timeseries[0]-ts[0]).days:]
s = s[:len(ts)]

print(len(symptomatic_timeseries),len(s))
for t,d in zip(symptomatic_timeseries,s):
    print(t,d)

sources["rki_symptomatic"] = (s[:len(symptomatic_timeseries)],symptomatic_timeseries)


150 169
2020-02-01 00:00:00 20.0
2020-02-02 00:00:00 25.0
2020-02-03 00:00:00 27.0
2020-02-04 00:00:00 29.0
2020-02-05 00:00:00 31.0
2020-02-06 00:00:00 32.0
2020-02-07 00:00:00 34.0
2020-02-08 00:00:00 34.0
2020-02-09 00:00:00 36.0
2020-02-10 00:00:00 42.0
2020-02-11 00:00:00 43.0
2020-02-12 00:00:00 47.0
2020-02-13 00:00:00 52.0
2020-02-14 00:00:00 58.0
2020-02-15 00:00:00 67.0
2020-02-16 00:00:00 72.0
2020-02-17 00:00:00 81.0
2020-02-18 00:00:00 91.0
2020-02-19 00:00:00 101.0
2020-02-20 00:00:00 128.0
2020-02-21 00:00:00 144.0
2020-02-22 00:00:00 167.0
2020-02-23 00:00:00 211.0
2020-02-24 00:00:00 276.0
2020-02-25 00:00:00 380.0
2020-02-26 00:00:00 533.0
2020-02-27 00:00:00 693.0
2020-02-28 00:00:00 901.0
2020-02-29 00:00:00 1089.0
2020-03-01 00:00:00 1304.0
2020-03-02 00:00:00 1587.0
2020-03-03 00:00:00 1862.0
2020-03-04 00:00:00 2238.0
2020-03-05 00:00:00 2647.0
2020-03-06 00:00:00 3281.0
2020-03-07 00:00:00 4081.0
2020-03-08 00:00:00 5166.0
2020-03-09 00:00:00 6853.0
2020-03-10 0

In [ ]:

startdate = datetime.date(2020,2,16)
ts = pd.date_range(startdate,periods=(datetime.date(2020,3,22)-startdate).days,freq='D')
daten200323_begin = {200216:2,200217:4,200218:2,200219:3,200220:10,200221:8,200222:10,200223:13,200224:41,200225:69,200226:113,200227:98,200228:126,200229:113,200301:119,200302:175,200303:169,200304:253,200305:243,200306:366,200307:493,200308:651,200309:831,200310:1044,200311:1264,200312:1199,200313:1218,200314:1089,200315:909,200316:825,200317:488,200318:307,200319:131,200320:40}
ddata = []
for t in ts:
    tstr = t.strftime("%y%0m%d")
    s = daten200323_begin.get(int(tstr),0)
    ddata.append(s)
data = np.cumsum(ddata)
sources["Datenstand-2020-03-22"] = (data,ts[:len(data)])

plt.plot(ts,data)

# Select Datasource
data,ts = sources["Datenstand-2020-03-22"]

In [5]:
# Select Datasource
data,ts = sources["rki_symptomatic"]

In [ ]:
fig = plt.figure(figsize=(8,6))
fig.subplots_adjust(wspace=0.33,hspace=0.27,left=0.05,right=0.98,top=0.94,bottom=0.05)
fig.suptitle("graphical overview of different sources")
gs = fig.add_gridspec(2, 1)

ax = fig.add_subplot(gs[0,0])
bx = fig.add_subplot(gs[1,0])

i = 0
color=["red","blue"]
keys = ["rki_symptomatic","Datenstand-2020-03-22"]
for k in keys:
    d,t = sources[k]
    ax.plot(t,d,'+',label=k,color=color[i])
    bx.plot(t[1:],np.diff(d),label=k,color=color[i])
    i+=1
    
ax.set_xlim([datetime.date(2020,2,21),datetime.date(2020,7,1)])
bx.set_xlim([datetime.date(2020,2,21),datetime.date(2020,7,1)])
ax.legend()
    
for tx in [ax,bx]:
    prec = 1.0 / (np.log10(tx.get_ylim()[1]) - 2.5)
    if prec < 2.0 and prec >= 0:
        tx.yaxis.set_major_formatter(
            matplotlib.ticker.FuncFormatter(_format_k(int(prec)))
        )
    tx.spines['top'].set_visible(False)
    tx.spines['right'].set_visible(False)
        

In [6]:
# Test simple SIR-Fit beta,gamma,N,I0

def FitSIRModel(data,l=90,initial={},I0=None,fit_differences=False):
    """
    Fit SIR model using HamiltonianMonteCarlo No-UTurn-Sampler 
        fit_0 : Allow fitting I0 else I0 = 1
        fit_differences : False # keep it that way until daily differences are of interest and not total numbers
    """
    with pm.Model() as model:
        # Setup Priors for beta,gamma
        beta = pm.Lognormal(name="beta",
                        mu=tt.log(tt.cast(.25,dtype='float64')),
                        sigma=tt.cast(.9,dtype='float64')) # Choose both beta and gamma wide enough
        gamma = pm.Lognormal(name="gamma",
                         mu=tt.log(tt.cast(.13,dtype='float64')),
                         sigma=tt.cast(.9,dtype='float64'))
        beta = tt.clip(beta,gamma,1e3) # Ensure positive initial growth, improves sampling
    
#        N = tt.cast(83e6,dtype='float64')
        N = pm.Lognormal(name="N",
                         mu=tt.log(tt.cast(200000,dtype='float64')),
                         sigma=tt.cast(.8,dtype='float64'))
        S0 = N
        if I0 == None: # Eiher fit or don't fit I0
            I0 = pm.Lognormal(name="I0",
                      mu=tt.log(tt.cast(20,dtype='float64')),
                      sigma=tt.cast(.7,dtype="float64")
                     )
        else:
            I0 = tt.cast(I0,dtype='float64')
    
        R0 = tt.cast(0.,dtype='float64')
    
        # Run the Model
        S_t,I_t,R_t,dS,dI,dR,eff = plain_SIR_model(beta,gamma,N,S0,I0,R0,l=l)
        # Store Model output in Trace
        pm.Deterministic("S_t",S_t)
        pm.Deterministic("I_t",I_t)
        pm.Deterministic("R_t",R_t)
        pm.Deterministic("dS",dS)
        pm.Deterministic("dR",dR)
        
        # Loglike-estimate
        ignore_front = 0 # 30
        if fit_differences: # Fit differences, which might me more suitable for the Sampler
            est = -dS
            sigma_obs = pm.HalfCauchy( name="sigma", beta=10 )
            pm.StudentT(
                    name="like_studentT",
                    nu=4,
                    mu=est,
                    sigma=tt.abs_(est + 1) ** 0.5 * sigma_obs,
                    observed=np.diff(data[:l+1])
            )
        else: # Leads to tighter fit to the non-differentiated data
#            est_deaths = R_t
            est = tt.cumsum(-dS)[ignore_front:]
            pm.Deterministic("est",est)
            sigma_obs = pm.HalfCauchy( name="sigma", beta=40 )
            pm.StudentT(
                    name="like_studentT",
                    nu=4,
                    mu=est,
                    sigma=tt.abs_(est + 1) ** 0.5 * sigma_obs,
                    observed=data[ignore_front:l]
            )
        
        # Handle Chained sampling reusing previous fit as starting-point
        init = "advi+adapt_diag" if len(initial) == 0 else None
        trace = pm.sample( model=model,init=init,start=initial,draws=1600,tune=2500,cores=4,chains=4)#,max_treedepth=16)
        return trace

In [ ]:
l = 80
l = len(ts)
print(l)
trace = FitSIRModel(data,l=l,I0=None)


In [ ]:
#l = 80

#with open("traces/sir_80","rb+") as f:
#    pickle.load(f)
 #   pickle.load(f)
 #   trace = pickle.load(f)

plt.plot(ts[:l],data[:l],"+")
#plt.plot(ts[:l],np.mean(trace.dI,axis=0))
plt.plot(ts[:l],np.mean(np.cumsum(-trace.dS,axis=1),axis=0),color="tab:red")
plt.fill_between(ts[:l],*np.percentile(np.cumsum(-trace.dS,axis=1),axis=0,q=(2.5,97.5)),alpha=.1,color="tab:red")
#plt.plot(ts[:l],np.mean(trace.est,axis=0))

plt.plot(ts[:l],np.mean(trace.I_t,axis=0),color="purple")
plt.fill_between(ts[:l],*np.percentile(trace.I_t,axis=0,q=(2.5,97.5)),alpha=.1,color="purple")

#plt.plot(ts[:l],np.mean(trace.S_t,axis=0),color="tab:blue")
#plt.fill_between(ts[:l],*np.percentile(trace.S_t,axis=0,q=(2.5,97.5)),color="tab:blue",alpha=.1)

plt.plot(ts[:l],np.mean(trace.R_t,axis=0),color="green")
plt.fill_between(ts[:l],*np.percentile(trace.R_t,axis=0,q=(2.5,97.5)),alpha=.1,color="green")

print(np.mean(trace.beta),np.percentile(trace.beta,q=(2.5,97.5)))
print(np.mean(trace.gamma),np.percentile(trace.gamma,q=(2.5,97.5)))
print(np.mean(trace.I0),np.percentile(trace.I0,q=(2.5,97.5)))
print(np.mean(trace.N),np.percentile(trace.N,q=(2.5,97.5)))

with open("traces/sir_%d_N200k"%len(ts),"wb+") as f:
    pickle.dump(ts,f)
    pickle.dump(data,f)
    pickle.dump(trace,f)

In [ ]:
plt.plot(ts[:l],data[:l],"+")
#plt.plot(ts[:l],np.mean(trace.dI,axis=0))
plt.plot(ts[:l],np.mean(np.cumsum(-trace.dS,axis=1),axis=0),color="tab:red")
plt.fill_between(ts[:l],*np.percentile(np.cumsum(-trace.dS,axis=1),axis=0,q=(2.5,97.5)),alpha=.1,color="tab:red")
#plt.plot(ts[:l],np.mean(trace.est,axis=0))

plt.plot(ts[:l],np.mean(trace.I_t,axis=0),color="purple")
plt.fill_between(ts[:l],*np.percentile(trace.I_t,axis=0,q=(2.5,97.5)),alpha=.1,color="purple")

plt.plot(ts[:l],np.mean(trace.S_t,axis=0),color="tab:blue")
plt.fill_between(ts[:l],*np.percentile(trace.S_t,axis=0,q=(2.5,97.5)),color="tab:blue",alpha=.1)

plt.plot(ts[:l],np.mean(trace.R_t,axis=0),color="green")
plt.fill_between(ts[:l],*np.percentile(trace.R_t,axis=0,q=(2.5,97.5)),alpha=.1,color="green")

print(np.mean(trace.beta),np.percentile(trace.beta,q=(2.5,97.5)))
print(np.mean(trace.gamma),np.percentile(trace.gamma,q=(2.5,97.5)))
#print(np.mean(trace.I0),np.percentile(trace.I0,q=(2.5,97.5)))
print(np.mean(trace.N),np.percentile(trace.N,q=(2.5,97.5)))


In [7]:
def SIRSeries(data,start=30,end=120,fit_I0=True):
    estimates = {}
    
    tr = None
    for l in range(start,end+1):
        initial = {} if tr == None else {'beta':np.mean(tr.beta),'gamma':np.mean(tr.gamma),'N':np.mean(tr.N)}
        
        if fit_I0 and tr != None:
            initial['I0'] = np.mean(tr.I0)
        print(l,initial)
        
#        initial=initial
        
        tr = FitSIRModel(data=data,l=l,I0=None)
        
        estimates[l] = {'beta':tr.beta,'gamma':tr.gamma,'N':tr.N,"est":np.cumsum(-tr.dS,axis=1)}
        if fit_I0:
            estimates[l]["I0"] = tr.I0
    return estimates

sir_est = SIRSeries(data,30,60)

30 {}


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '3522' (I am process '4694')
Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 4500
Interrupted at 4,499 [2%]: Average Loss = 221.28
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, I0, N, gamma, beta]


Sampling 4 chains for 2_500 tune and 1_600 draw iterations (10_000 + 6_400 draws total) took 183 seconds.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6710268494300792, but should be close to 0.8. Try to increase the number of tuning steps.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6697314227075737, but should be close to 0.8. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5957880296581041, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


31 {'beta': 0.24727795960639085, 'gamma': 0.06172536175557521, 'N': 289460.001596363, 'I0': 5.162817611728303}


...

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, I0, N, gamma, beta]


Sampling 4 chains for 2_500 tune and 1_600 draw iterations (10_000 + 6_400 draws total) took 827 seconds.
The acceptance probability does not match the target. It is 0.6477206558186342, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7017265883112518, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.5693902410903557, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.616216630850906, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


60 {'beta': 0.3072663095852464, 'gamma': 0.04996261442514831, 'N': 80525.09248394571, 'I0': 1.1897696082857894}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, I0, N, gamma, beta]


Sampling 4 chains for 2_500 tune and 1_600 draw iterations (10_000 + 6_400 draws total) took 956 seconds.
The acceptance probability does not match the target. It is 0.572779521603437, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [8]:
with open("estimates/sir_symptomatic_%d"%time.time(),"wb+") as f:
    pickle.dump(sir_est,f)

for k,v in sir_est.items():
#    print(k,v["ssd"])
    for p in ['beta','gamma','N','I0']:
        print("\t",p,np.mean(v[p]),np.percentile(v[p],q=(2.5,5,50,95,97.5)))

	 beta 0.24727795960639085 [0.18655331 0.19377147 0.2437316  0.31385935 0.32139477]
	 gamma 0.06172536175557521 [0.01371915 0.01744446 0.05498513 0.12689899 0.13623394]
	 N 289460.001596363 [ 47454.89398334  56881.01018459 210340.80858579 750417.25347316
 917841.81030194]
	 I0 5.162817611728303 [2.52787669 2.94258297 4.79360367 8.51442654 9.68924375]
	 beta 0.24931714750998743 [0.19741546 0.20414318 0.24596159 0.30407129 0.31463102]
	 gamma 0.05423386270238229 [0.01244079 0.01576821 0.04971782 0.10624961 0.11851313]
	 N 270839.6023171962 [ 45172.28731643  58283.96310994 203921.23398763 675128.46486493
 904233.20646955]
	 I0 4.553093393613878 [2.35439504 2.62690311 4.36630429 7.24745752 7.96420807]
	 beta 0.2573789036173357 [0.20548368 0.21094811 0.25359678 0.31151179 0.32259195]
	 gamma 0.056759172622979005 [0.01292806 0.01662757 0.05189735 0.10872794 0.11789583]
	 N 268868.7933120525 [ 46723.59570883  56180.36433577 198850.4655444  705925.01793894
 996447.40890339]
	 I0 4.107686408350

In [9]:
def FitGompertz(data,l=90,initial={}):
    with pm.Model() as model:
        a = pm.Lognormal(name="a",
                    mu=tt.log(tt.cast(1e5,dtype='float64')),
                    sigma=tt.cast(.6,dtype='float64'))
        b = pm.Lognormal(name="b",
                    mu=tt.log(tt.cast(11,dtype='float64')),
                    sigma=tt.cast(.8,dtype='float64'))
        c = pm.Lognormal(name="c",
                    mu=tt.log(tt.cast(.07,dtype='float64')),
                    sigma=tt.cast(.8,dtype='float64'))
    
        est = Gompertz(a,b,c,l)
        est_deaths = pm.Deterministic("deaths",est)
    
        sigma_obs = pm.HalfCauchy( name="sigma", beta=40 )
        pm.StudentT(
                    name="like_studentT",
                    nu=4,
                    mu=est_deaths,
                    sigma=tt.abs_(est_deaths + 1) ** 0.5 * sigma_obs,
                    observed=data[:l]
        )
        init = "advi+adapt_diag" if len(initial) == 0 else None
        trace = pm.sample( model=model,init=init,start=initial,draws=1200,tune=2000,cores=4,chains=4)
        return trace

In [ ]:
l=len(ts)
trace = FitGompertz(data,l)

In [ ]:
#l=80
#gtrace = trace
plt.plot(ts[:l],data[:l],"+")
#plt.plot(ts[:l],np.mean(trace.dI,axis=0))
plt.plot(ts[:l],np.mean(gtrace.deaths,axis=0))

print(np.median(gtrace.a),np.percentile(trace.a,q=(2.5,97.5)))
print(np.median(gtrace.b),np.percentile(trace.b,q=(2.5,97.5)))
print(np.median(gtrace.c),np.percentile(trace.c,q=(2.5,97.5)))

print( np.sum(np.power(data[:l]-np.mean(gtrace.deaths[:l],axis=0),2 ) ) )


In [10]:
def GompertzSeries(data,start=30,end=120):
    estimates = {}
    tr = None
    for l in range(start,end+1):
        initial = {} if tr == None else {'a':np.mean(tr.a),'b':np.mean(tr.b),'c':np.mean(tr.c)}
        print(l,initial)
        tr = FitGompertz(data,l=l)#,initial=initial)
#        ssd = np.sum(np.power(data[:l]-np.mean(tr.deaths,axis=0),2))
        estimates[l] = {'a':tr.a,'b':tr.b,'c':tr.c,"est":tr.deaths}
    return estimates

gomp_est = GompertzSeries(data,30,60)


30 {}


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 11000
Interrupted at 10,999 [5%]: Average Loss = 222.93
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, c, b, a]


Sampling 4 chains for 2_000 tune and 1_200 draw iterations (8_000 + 4_800 draws total) took 20 seconds.
The number of effective samples is smaller than 25% for some parameters.


31 {'a': 159998.165886908, 'b': 10.9410281230157, 'c': 0.027141995447891056}


...

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, c, b, a]


Sampling 4 chains for 2_000 tune and 1_200 draw iterations (8_000 + 4_800 draws total) took 25 seconds.
The number of effective samples is smaller than 25% for some parameters.


60 {'a': 214540.92653439613, 'b': 29.668793403463024, 'c': 0.06063049386576329}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, c, b, a]


Sampling 4 chains for 2_000 tune and 1_200 draw iterations (8_000 + 4_800 draws total) took 24 seconds.
The acceptance probability does not match the target. It is 0.681423273945523, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


In [11]:
with open("estimates/gomp_symptomatic_%d"%time.time(),"wb+") as f:
    pickle.dump(gomp_est,f)

for k,v in gomp_est.items():
    print(k,v["ssd"])
    for p in ['a','b','c']:
        print("\t",p,np.mean(v[p]),np.percentile(v[p],q=(2.5,5,50,95,97.5)))


KeyError: 'ssd'

In [ ]:
trace = FitGompertz(l=30)
plt.plot(ts1[:l],deaths1[:l],"+")
plt.plot(ts1[:l],trace.deaths[0])
plt.fill_between(ts1[:l],*np.percentile(trace.deaths,axis=0,q=(2.5,97.5)),color="orange",alpha=.2)

gomp_est = np.mean(trace.deaths,axis=0)
print(SSD(deaths1[:1],gomp_est))


In [ ]:
a = [x for x in range(10)]
print(a[7:3])